In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import tqdm_notebook as tqdm

In [2]:
def calCorrelation(a, b, windowSize, stepSize):
    def calTempCorrelation(a, b):
        numerator = 0
        dedominator = 0
        tempA = 0
        tempB = 0
        for i, j in zip(a, b):
            numerator += np.abs(i)*j
            tempA += i*i
            tempB += j*j
        dedominator = np.sqrt(tempA*tempB)
        return numerator/dedominator
    corr = []
    for i in range(0, len(a), stepSize):
        corr.append(calTempCorrelation(a[i: i+windowSize], b[i: i+windowSize]))
    return corr

In [3]:
def activitySplit(csvName, windowSize, stepSize, corrThresholdLength):
    joinWindowSize = windowSize  # 两个窗口拼接成一个时间的阈值
    folderPath = r"E:\Jupyter\实验室\采集数据\yqcc"
    csvPath = folderPath + "\\" + csvName
    dataXYZ = pd.read_csv(csvPath, header= 0)
    corrXYs = calCorrelation(dataXYZ.iloc[:, 0], dataXYZ.iloc[:, 1], windowSize, stepSize) 
    activityPosList = []
    isFirst = True
    
    corrThresholdList = corrXYs[: corrThresholdLength]
    corrThresholdListCopy = corrThresholdList.copy().sort()
    percent25 = corrThresholdList[int(0.25*corrThresholdLength)]
    percent75 = corrThresholdList[int(0.75*corrThresholdLength)]
    
    coef = 30
    corrThreshold = np.abs(percent75 + coef*(np.abs(percent75 - percent25)))
    
    for i, corrXY in enumerate(corrXYs):
        if np.abs(corrXY) <= corrThreshold:
            if isFirst:
                start = i * stepSize
                end = start + windowSize
                isFirst = False
            else:
                if i*stepSize - end <= joinWindowSize:
                    end = i*stepSize + windowSize
                else:
                    activityPosList.append(str(start) + " " + str(end))
                    start = i * stepSize
                    end = start + windowSize
        corrThresholdList.pop(0)
        corrThresholdList.append(corrXY)
        corrThresholdListCopy = corrThresholdList.copy().sort()
        percent25 = corrThresholdList[int(0.25*corrThresholdLength)]
        percent75 = corrThresholdList[int(0.75*corrThresholdLength)]
        corrThreshold = np.abs(percent75 + coef*(np.abs(percent75 - percent25)))

    print(csvName, len(activityPosList))
    return len(activityPosList)
    


In [4]:
%%time
csvPath = r"E:\Jupyter\实验室\采集数据\yqcc"
csvLists = os.listdir(csvPath)

for csvName in csvLists:
    activitySplit(csvName, 32, 16, 500)
    

dig_1.0_A.csv 54
dig_1.0_B.csv 50
dig_1.0_C.csv 52
dig_1.0_D.csv 47
dig_1.0_E.csv 52
dig_1.0_F.csv 43
dig_2.0_A.csv 47
dig_2.0_B.csv 35
dig_2.0_C.csv 44
dig_2.0_D.csv 49
dig_2.0_E.csv 46
dig_2.0_F.csv 47
dig_3.0_A.csv 49
dig_3.0_B.csv 51
dig_3.0_C.csv 37
dig_3.0_D.csv 47
dig_3.0_E.csv 39
dig_3.0_F.csv 38
dig_4.0_A.csv 42
dig_4.0_B.csv 56
dig_4.0_C.csv 23
dig_4.0_D.csv 47
dig_4.0_E.csv 26
dig_4.0_F.csv 36
dig_5.0_A.csv 40
dig_5.0_B.csv 31
dig_5.0_C.csv 20
dig_5.0_D.csv 22
dig_5.0_E.csv 22
dig_5.0_F.csv 26
dig_6.0_A.csv 32
dig_6.0_B.csv 24
dig_6.0_C.csv 27
dig_6.0_D.csv 39
dig_6.0_E.csv 17
dig_6.0_F.csv 22
dig_7.0_A.csv 35
dig_7.0_B.csv 22
dig_7.0_C.csv 10
dig_7.0_D.csv 37
dig_7.0_E.csv 11
dig_7.0_F.csv 14
dig_8.0_A.csv 36
dig_8.0_B.csv 26
dig_8.0_C.csv 13
dig_8.0_D.csv 45
dig_8.0_E.csv 11
dig_8.0_F.csv 13
jumpVarity_A.csv 23
jump_1.0_A.csv 57
jump_1.0_B.csv 70
jump_1.0_C.csv 50
jump_1.0_D.csv 46
jump_1.0_E.csv 59
jump_1.0_F.csv 51
jump_2.0_A.csv 56
jump_2.0_B.csv 55
jump_2.0_C.csv 52
ju

In [43]:
def calMean(a, windowSize, stepSize):
    mean = []
    for i in range(0, len(a), stepSize):
        mean.append(np.mean(a[i, i+windowSize]))
    return mean


In [49]:
def activitySplitByMean(csvName, windowSize, stepSize, meanThresholdLength):
    joinWindowSize = windowSize  # 两个窗口拼接成一个时间的阈值
    folderPath = r"E:\Jupyter\实验室\采集数据\yqcc"
#     txtFolderPath = r"E:\Jupyter\实验室\采集数据\data_syf"
    csvPath = folderPath + "\\" + csvName
#     txtName = csvName[0:-3] + "txt"
#     txtPath = txtFolderPath + "\\" + txtName
    
    # 标记的数据
#     try:
#         txtFile = open(txtPath, "r")
#         txtDataList = []
#         for line in txtFile.readlines():
#             txtDataList.append(line.strip())
#         txtFile.close()
#     except Exception as e:
#         print(e)
#         return
    # print(txtDataList)
#     print(csvPath)
    df = pd.read_csv(csvPath, header= 0)
#     meanXYs = calMean(df.iloc[:, 0], windowSize, stepSize) 
    df.columns = [0,1,2]
    
    temp = 0
    event_seg = []
    count = 0
    num = 0
    for i,j in enumerate(df[0]):

        if i < 500:
            base_std = 0.0003413866873133119
            base_mean = np.mean(df[0][0:100])
        else:
            #base_std = np.array(df[0][(i - 500 + 1):(i + 1)]).std()
            base_mean = np.array(df[0][(i - 500 + 1):(i + 1)]).mean()



        if (base_mean - 3 *  base_std > j) | (base_mean + 3 *  base_std < j):


            if i - temp > 100:

                #plt.figure()
                count = count + 1
                '''0721添加
                print('base_mean', count, ':',base_mean)
                print('base_std:', count, ':',base_std)
                '''
                temp = i
                #print(temp)
                c = list(df[0][(i - 50):(i + 50)])
                max_temp_index = c.index(max(c))
                min_temp_index = c.index(min(c))
                index_med = int((max_temp_index + min_temp_index) / 2)
                #print(max_temp_index)
                max_temp = np.max(c)
                #plt.plot(df[0][((temp - 50) + index_med - 50) : ((temp - 50) + index_med + 50)])
                #plt.title(str(count-1))
                num += 1
#                 event_seg.append(df[0][((temp - 50) + index_med - 50) : ((temp - 50) + index_med + 50)])
    print(csvName, num)
    return num

In [50]:
%%time
csvPath = r"E:\Jupyter\实验室\采集数据\yqcc"
csvLists = os.listdir(csvPath)

for csvName in csvLists:
    activitySplitByMean(csvName, 16, 8, 100)

dig_1.0_A.csv 49
dig_1.0_B.csv 53
dig_1.0_C.csv 55
dig_1.0_D.csv 50
dig_1.0_E.csv 67
dig_1.0_F.csv 50
dig_2.0_A.csv 49
dig_2.0_B.csv 50
dig_2.0_C.csv 49
dig_2.0_D.csv 50
dig_2.0_E.csv 50
dig_2.0_F.csv 50
dig_3.0_A.csv 49
dig_3.0_B.csv 50
dig_3.0_C.csv 45
dig_3.0_D.csv 49
dig_3.0_E.csv 38
dig_3.0_F.csv 48
dig_4.0_A.csv 45
dig_4.0_B.csv 41
dig_4.0_C.csv 11
dig_4.0_D.csv 52
dig_4.0_E.csv 36
dig_4.0_F.csv 32
dig_5.0_A.csv 23
dig_5.0_B.csv 21
dig_5.0_C.csv 16
dig_5.0_D.csv 35
dig_5.0_E.csv 18
dig_5.0_F.csv 14
dig_6.0_A.csv 18
dig_6.0_B.csv 1
dig_6.0_C.csv 4
dig_6.0_D.csv 27
dig_6.0_E.csv 0
dig_6.0_F.csv 4
dig_7.0_A.csv 19
dig_7.0_B.csv 1
dig_7.0_C.csv 0
dig_7.0_D.csv 11
dig_7.0_E.csv 5
dig_7.0_F.csv 4
dig_8.0_A.csv 20
dig_8.0_B.csv 0
dig_8.0_C.csv 1
dig_8.0_D.csv 28
dig_8.0_E.csv 0
dig_8.0_F.csv 0
jumpVarity_A.csv 19
jump_1.0_A.csv 50
jump_1.0_B.csv 50
jump_1.0_C.csv 52
jump_1.0_D.csv 50
jump_1.0_E.csv 50
jump_1.0_F.csv 50
jump_2.0_A.csv 50
jump_2.0_B.csv 50
jump_2.0_C.csv 53
jump_2.0_D.csv

In [91]:
# activitySplit(csvLists[0], 10, 5, 0.9995)

In [92]:
# activitySplit("walk_A_0.5_normal.csv", 10, 5, 0.9999)

In [100]:
txtLenAll = []
activityNumAll = []
recallAll = []
precisionAll = []
f1scoreAll = []
truePosTestPosAll = []
testPosTruePosAll = []
for csvName in tqdm(csvLists):
    if csvName[0:4] !="zuan":
        txtLen, activityNum, truePosTestPos, testPosTruePos, recall, precision, f1score = activitySplit(csvName, 10, 5, 0.9999)
        txtLenAll.append(txtLen)
        activityNumAll.append(activityNum)
        truePosTestPosAll.append(truePosTestPos)
        testPosTruePosAll.append(testPosTruePos)
        recallAll.append(recall)
        precisionAll.append(precision)
        f1scoreAll.append(f1score)

dig_A_0.5.csv 1.0 0.8620689655172413 0.9259259259259259
dig_A_1.0.csv 0.9803921568627451 1.0 0.99009900990099
dig_A_1.5.csv 0.9807692307692307 0.9272727272727272 0.9532710280373831
dig_A_2.0.csv 0.9807692307692307 0.9444444444444444 0.9622641509433962
dig_A_2.5.csv 0.98 0.9074074074074074 0.9423076923076924
dig_A_3.0.csv 1.0 0.9259259259259259 0.9615384615384615
dig_A_3.5.csv 0.9795918367346939 0.9230769230769231 0.9504950495049506
dig_A_4.0.csv 0.9814814814814815 0.9814814814814815 0.9814814814814815
dig_A_4.5.csv 0.9807692307692307 0.9807692307692307 0.9807692307692307
dig_A_5.0.csv 0.9811320754716981 1.0 0.9904761904761905
dig_A_5.5.csv 0.9803921568627451 0.9615384615384616 0.970873786407767
dig_A_6.0.csv 0.9607843137254902 0.98 0.9702970297029702
dig_A_6.5.csv 0.98 0.9607843137254902 0.9702970297029702
dig_A_7.0.csv 0.9803921568627451 0.9615384615384616 0.970873786407767
dig_A_7.5.csv 0.9782608695652174 0.9375 0.9574468085106383
dig_B_0.5.csv 1.0 0.78125 0.8771929824561403
dig_B_1.

In [124]:
# 所有
# A， B
# dig所有
# jump
trainAllNum = 0
activityAllNum = 0
truePosTestPosAllNum = 0
testPosTruePosAllNum = 0

trainANum = 0
activityANum = 0
truePosTestPosANum = 0
testPosTruePosANum = 0

trainBNum = 0
activityBNum = 0
truePosTestPosBNum = 0
testPosTruePosBNum = 0


trainDigNum = 0
activityDigNum = 0
truePosTestPosDigNum = 0
testPosTruePosDigNum = 0

trainJumpNum = 0
activityJumpNum = 0
truePosTestPosJumpNum = 0
testPosTruePosJumpNum = 0

trainWalkHardNum = 0
activityWalkHardNum = 0
truePosTestPosWalkHardNum = 0
testPosTruePosWalkHardNum = 0

trainWalkNormalNum = 0
activityWalkNormalNum = 0
truePosTestPosWalkNormalNum = 0
testPosTruePosWalkNormalNum = 0

for i, csvName in tqdm(enumerate(csvLists[:-2])):
    trainAllNum += txtLenAll[i]
    activityAllNum += activityNumAll[i]
    truePosTestPosAllNum += truePosTestPosAll[i]
    testPosTruePosAllNum += testPosTruePosAll[i]
    if csvName.split("_")[1] == "A":
        trainANum += txtLenAll[i]
        activityANum += activityNumAll[i]
        truePosTestPosANum += truePosTestPosAll[i]
        testPosTruePosANum += testPosTruePosAll[i]
    else:
        trainBNum += txtLenAll[i]
        activityBNum += activityNumAll[i]
        truePosTestPosBNum += truePosTestPosAll[i]
        testPosTruePosBNum += testPosTruePosAll[i]
    if csvName.split("_")[0] == "dig":
        trainDigNum += txtLenAll[i]
        activityDigNum += activityNumAll[i]
        truePosTestPosDigNum += truePosTestPosAll[i]
        testPosTruePosDigNum += testPosTruePosAll[i]
    elif csvName.split("_")[0] == "jump":
        trainJumpNum += txtLenAll[i]
        activityJumpNum += activityNumAll[i]
        truePosTestPosJumpNum += truePosTestPosAll[i]
        testPosTruePosJumpNum += testPosTruePosAll[i]
    elif csvName.split("_")[-1] == "hard.csv":
        trainWalkHardNum += txtLenAll[i]
        activityWalkHardNum += activityNumAll[i]
        truePosTestPosWalkHardNum += truePosTestPosAll[i]
        testPosTruePosWalkHardNum += testPosTruePosAll[i]
    elif csvName.split("_")[-1] == "normal.csv":
        trainWalkNormalNum += txtLenAll[i]
        activityWalkNormalNum += activityNumAll[i]
        truePosTestPosWalkNormalNum += truePosTestPosAll[i]
        testPosTruePosWalkNormalNum += testPosTruePosAll[i]

xxx


'normal.txt'

In [126]:
recallAll = truePosTestPosAllNum / trainAllNum
precisionAll = testPosTruePosAllNum / activityAllNum
f1scoreAll = 2 * recallAll * precisionAll / (recallAll + precisionAll)
print(recallAll, precisionAll, f1scoreAll)


recallA = truePosTestPosANum / trainANum
precisionA = testPosTruePosANum / activityANum
f1scoreA = 2 * recallA * precisionA / (recallA + precisionA)
print(recallA, precisionA, f1scoreA)

recallB = truePosTestPosBNum / trainBNum
precisionB = testPosTruePosBNum / activityBNum
f1scoreB = 2 * recallB * precisionB / (recallB + precisionB)
print(recallB, precisionB, f1scoreB)

recallDig = truePosTestPosDigNum / trainDigNum
precisionDig = testPosTruePosDigNum / activityDigNum
f1scoreDig = 2 * recallDig * precisionDig / (recallDig + precisionDig)
print(recallDig, precisionDig, f1scoreDig)

recallJump = truePosTestPosJumpNum / trainJumpNum
precisionJump = testPosTruePosJumpNum / activityJumpNum
f1scoreJump = 2 * recallJump * precisionJump / (recallJump + precisionJump)
print(recallJump, precisionJump, f1scoreJump)

recallWalkHard = truePosTestPosWalkHardNum / trainWalkHardNum
precisionWalkHard = testPosTruePosWalkHardNum / activityWalkHardNum
f1scoreWalkHard = 2 * recallWalkHard * precisionWalkHard / (recallWalkHard + precisionWalkHard)
print(recallWalkHard, precisionWalkHard, f1scoreWalkHard)


recallWalkNormal = truePosTestPosWalkNormalNum / trainWalkNormalNum
precisionWalkNormal = testPosTruePosWalkNormalNum / activityWalkNormalNum
f1scoreWalkNormal = 2 * recallWalkNormal * precisionWalkNormal / (recallWalkNormal + precisionWalkNormal)
print(recallWalkNormal, precisionWalkNormal, f1scoreWalkNormal)

0.9668109668109668 0.9039395574743659 0.9343187839910752
0.9707773232028054 0.8840425531914894 0.9253819999523998
0.9629418472063854 0.9244249726177437 0.9432903889019211
0.9807564698075647 0.9566343042071197 0.9685452162516381
0.9701280227596017 0.9394773039889959 0.9545566788904908
0.9641294838145232 0.8404580152671756 0.8980560408843169
0.7857142857142857 0.7096774193548387 0.7457627118644068


In [103]:
len(txtLenAll)

86

In [104]:
len(csvLists)

88